# Annotation errors hava found:

- doc_50: full anned as FM
- doc_169: first char missing at idx 10
- doc_59: 3 lost last

In [1]:
from os import listdir

directory  = './Data/bioc-FH-training/'
file_list  = listdir(directory)
file_names = set([file.split('.')[0] for file in file_list if file != '' and file.endswith('txt')])

# Read files

In [2]:
import codecs
#import chardet # chardet.detect(binary_var)['encoding']

texts = dict()

for file_name in file_names:
    file = directory + file_name + '.txt'
    
    with open(file, 'r') as f:
        texts[file_name] = f.read().replace('\n', ' ')

# Tokenization and POS

In [3]:
import nltk

sents  = dict()
tokens = dict()
pos    = dict()

for file_name in file_names:
    tokens_   = []
    tags      = []
    
    sentences = nltk.sent_tokenize(texts[file_name])
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        
        tokens_.append(words)
        tags.append(nltk.pos_tag(words))

    sents[file_name] = sentences
    tokens[file_name] = tokens_
    pos[file_name] = tags

In [4]:
# print(tokens)

In [5]:
# print(pos)

# Get annotated entities

In [6]:
import xml.etree.ElementTree

entities = {}
for file_name in file_names:
    file_path = directory + file_name + '.xml'
    
    root = xml.etree.ElementTree.parse(file_path).getroot()
    annotations = root.findall('annotations')[0]

    entities_ = [] # get span of every annoted entity
    for entity in annotations.findall('entity'):
        id = entity.find('id').text
        typee = entity.find('type')

        if typee.text in ['FamilyMember', 'Observation', 'LivingStatus']: # other options: Age, PHI
            properties = entity.find('properties')
            for spans in entity.findall('span')[0].text.split(';'):
                span = list(map(int, spans.split(',')))
                attributes = {}
                
                if typee.text == 'FamilyMember':
                    attributes['SideOfFamily'] = properties.find('SideOfFamily').text
                elif typee.text == 'Observation':
                    attributes['Negation'] = properties.find('Negation').text
                elif typee.text == 'LivingStatus':
                    attributes['Alive'] = properties.find('Alive').text
                    attributes['Healthy'] = properties.find('Healthy').text
                    attributes['ID'] = properties.find('ID').text
                
                entities_.append((span[0], span[1],
                                  typee.text,
                                  texts[file_name][span[0]:span[1]],
                                  attributes,
                                  id))

    entities_.sort(key=lambda tup: tup[1])
    entities[file_name] = entities_

In [ ]:
num_entities = 0
living_status = []
for file_name in file_names:
    print('\n', file_name)
#     print(texts[file_name])
    for entity in entities[file_name]:
        num_entities += 1
        if entity[2] == 'LivingStatus':
            living_status.append(entity[3])
        print(entity)

In [8]:
from collections import Counter
Counter(living_status)

Counter({'died': 123,
         'passed away': 11,
         'alive': 7,
         'well': 5,
         'healthy': 118,
         'good health': 32,
         'health': 2,
         'died ': 39,
         'ied ': 1,
         'living': 5,
         'healthy ': 10,
         'good general health': 11,
         'passed away ': 17,
         'living and well': 7,
         ' living and well': 3,
         ' healthy': 2,
         'alive ': 4,
         'deceased': 3,
         ' alive': 1,
         'dead': 2,
         'alive and well': 1,
         ' health': 1,
         'doing well': 1,
         'generally healthy': 1,
         'health ': 1,
         'living ': 1,
         'without problems': 1,
         'good general health ': 1,
         'otherwise healthy': 1,
         'stillborn': 1,
         'death': 1,
         'living and well ': 1})

In [9]:
print("there are", num_entities, "enties in all.")

there are 2202 enties in all.


# Store to pickles

In [10]:
import pickle
from os.path import join

OBJECTS = './objects/'
with open(join(OBJECTS, "texts.pkl"), 'wb') as pkl_file:
    pickle.dump(texts, pkl_file)
with open(join(OBJECTS, "sents.pkl"), 'wb') as pkl_file:
    pickle.dump(sents, pkl_file)
with open(join(OBJECTS, "tokens.pkl"), 'wb') as pkl_file:
    pickle.dump(tokens, pkl_file)
with open(join(OBJECTS, "pos.pkl"), 'wb') as pkl_file:
    pickle.dump(pos, pkl_file)
with open(join(OBJECTS, "entities.pkl"), 'wb') as pkl_file:
    pickle.dump(entities, pkl_file)